In [25]:
import cv2
from random import randint
import numpy as np
from numpy.linalg import norm
from functools import reduce

In [3]:
def show(x, name):
    cv2.imshow(name, x)
    print(cv2.waitKey(0))
    cv2.destroyAllWindows()

def sekip(x):
    pass

In [216]:
!ls images

box.jpeg   car.jpeg	      pentagon.jpg  rectangle.jpg
car2.webp  elipse_circle.png  rect2.png     test_shape.jpg


In [257]:
def sudutTigaTitik(a, b, c):
    # Return sudut antara garis a-b dan b-c
    vAB = b - a
    vBC = b - c
    
    dot = np.dot(vAB, vBC)
    dAB = norm(vAB)
    dBC = norm(vBC)
    
    return np.rad2deg(np.arccos(dot/(dAB*dBC)))

def createTrackbar():
    cv2.namedWindow('setting')
    cv2.createTrackbar('d', 'setting', 14, 50, sekip)
    cv2.createTrackbar('sigmaColor', 'setting', 93, 200, sekip)
    cv2.createTrackbar('sigmaSpace', 'setting', 71, 200, sekip)
    cv2.createTrackbar('kSize', 'setting', 6, 10, sekip)
    cv2.createTrackbar('thres', 'setting', 47, 255, sekip)

def preProcImg(img):
    sigma = 0.33
    pp = img.copy()

    #Smoothing Image
    d = cv2.getTrackbarPos('d', 'setting')+1
    sigmaColor = cv2.getTrackbarPos('sigmaColor', 'setting')
    sigmaSpace =cv2.getTrackbarPos('sigmaSpace', 'setting')
    pp = cv2.bilateralFilter(pp, d, sigmaColor, sigmaSpace)
    pp = cv2.cvtColor(pp, cv2.COLOR_BGR2GRAY)
    
    #Morphology Transformation
    kSize = cv2.getTrackbarPos('kSize', 'setting')
    kernel = np.ones((kSize, kSize), np.uint8)
    pp = cv2.morphologyEx(pp, cv2.MORPH_GRADIENT, kernel)
    
    #Thresholding
    thres = cv2.getTrackbarPos('thres', 'setting')
    pp = cv2.threshold(pp,thres,255,cv2.THRESH_BINARY_INV)
    pp = cv2.bitwise_not(pp[1])
    
    # Apply Canny
    v = np.median(pp)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    pp = cv2.Canny(pp, lower, upper)
    pp = cv2.copyTo(img, pp)
    pp = cv2.cvtColor(pp, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Pre-Proc', pp)
    return pp

In [261]:
img = cv2.imread('images/test_shape.jpg')
img = cv2.resize(img, (800, 800))

createTrackbar()
while True:
    ctr = img.copy()
    
    pp = preProcImg(img)
    
    #Contour detection
    con, tree = cv2.findContours(pp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    conArea = [(i,cv2.contourArea(c)) for i,c in enumerate(con)]
    conArea.sort(key=lambda x : x[1], reverse=True)
    maxNumContour = 30
    eps = 30
    epsTitik = 30
        
    sudutPoly = []
    for i in range(min(len(conArea), maxNumContour)):
        color = (randint(0, 255), randint(0, 255), randint(0, 255))
        ctr = cv2.drawContours(ctr, con, conArea[i][0], color, 3)
#         has = simplifikasiTitik(DouglasPeucker(con[conArea[i][0]], eps), epsTitik)
        approx = cv2.approxPolyDP(con[conArea[i][0]], 0.01*cv2.arcLength(con[conArea[i][0]], True), True)
        for a in approx:
            x,y = a[0]
            ctr = cv2.circle(ctr, (x, y), 3, (0, 0, 255), 3)
        if(len(approx) > 2):
            sudut = []
            for i in range(len(approx)-2):
                p1 = approx[i][0]
                p2 = approx[i+1][0]
                p3 = approx[i+2][0]
                sudut.append(sudutTigaTitik(p1, p2, p3))
            sudut.append(sudutTigaTitik(approx[i][0], approx[i+1][0], approx[0][0]))
            sudutPoly.append((conArea[i][0], sudut))
    cv2.imshow('Contour', ctr)
    key = cv2.waitKey(1)
    if(key == ord('q')):
        break
    elif(key == ord('n')):
        continue
cv2.destroyAllWindows()

/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in arccos
  # Remove the CWD from sys.path while we load stuff.
/home/aditya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [259]:
sudutPoly

[(453,
  [108.30512477127822,
   91.30195267257888,
   7.870716794399954,
   168.13030936165322,
   136.1160103914073,
   123.28040090500583]),
 (313,
  [159.69843943725655,
   138.1908868120298,
   172.2866590415682,
   68.96248897457819,
   21.21707514313354]),
 (253,
  [90.0,
   116.56505117707799,
   53.13010235415599,
   150.2551187030578,
   130.36453657309735,
   133.09084756700364,
   149.03624346792648,
   81.2538377374448,
   129.70991879462875,
   149.03624346792648,
   155.5560452195835,
   160.31968691030963,
   118.92642583525362,
   145.49147701233161]),
 (254,
  [135.0,
   108.43494882292202,
   103.67130713219584,
   160.31968691030963,
   155.5560452195835,
   135.0,
   141.34019174590992,
   91.78991060824607,
   140.9061411137705,
   131.9335144988741,
   126.02737338510362,
   153.434948822922,
   71.56505117707799,
   135.0,
   84.8055710922652]),
 (313,
  [154.87978825768568,
   43.13232116056583,
   155.24662007558823,
   165.40162729814705,
   158.8079908199473

In [9]:
cv2.destroyAllWindows()

In [234]:
def DouglasPeucker(titik, eps):
    dmax = 0
    index = 0
    end = len(titik)
    pEnd = titik[end-1][0]
    pStart = titik[0][0]
    for i in range(2, end-1):
        p = titik[i][0]
        d = norm(np.cross(pEnd-pStart, pStart-p))/norm(pEnd-pStart)
        if(d > dmax):
            index = i
            dmax = d
    hasil = []
    if(dmax > eps):
#         Rekursif
        hasil1 = DouglasPeucker(titik[:index], eps)
        hasil2 = DouglasPeucker(titik[index:], eps)
        hasil = hasil1 + hasil2
    else:
        hasil = [pStart, pEnd]
    return hasil

def simplifikasiTitik(has, epsTitik):
    delSoon = set([])
    for i, h in enumerate(has):
        for j, cek in enumerate(has):
            if(norm(cek-h) < epsTitik and i < j):
                delSoon.update({j})
    titikSimple = [h for idx, h in enumerate(has) if idx not in delSoon]
    return titikSimple

In [238]:
p1 = np.array([0, 0])
p2 = np.array([1, 0])
p3 = np.array([2, -1])
print(sudutTigaTitik(p1, p2, p3))

45.00000000000001


In [246]:
approx

array([[[505,  78]],

       [[499,  78]],

       [[497,  79]],

       [[495,  83]],

       [[495,  87]],

       [[491,  91]],

       [[491, 100]],

       [[501, 100]],

       [[502,  99]],

       [[502,  93]],

       [[507,  91]],

       [[508,  89]],

       [[508,  80]]], dtype=int32)

In [255]:
sudutTigaTitik(approx[0][0], approx[1][0], approx[2][0] )

6.0   2.23606797749979   -12


153.434948822922

In [250]:
12 / (6 * 2.24)

0.8928571428571428

In [251]:
np.arccos(0.892)

0.4690458582650856

In [252]:
np.rad2deg(0.469)

26.871720591635608

In [260]:
img = cv2.imread('images/car.jpeg')
img = cv2.resize(img, (800, 800))
ctr = cv2.drawContours(img, con, 634, color, 3)
show(ctr, 'o')

113
